In [17]:
import pandas as pd
import numpy as np
import pickle

from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error

import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

import mlflow

from prefect import flow



def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

def add_features(train_filename: str, val_filename: str):
    df_train = read_dataframe(train_filename)
    df_val = read_dataframe(val_filename)
    
    print(f"Training data: {len(df_train)}\n Testing data: {len(df_val)}")
    
    # Add trip route as new category
    df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
    df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']


    categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
    numerical = ['trip_distance']

    dv = DictVectorizer()

    train_dicts = df_train[categorical + numerical].to_dict(orient='records')
    X_train = dv.fit_transform(train_dicts)

    val_dicts = df_val[categorical + numerical].to_dict(orient='records')
    X_val = dv.transform(val_dicts)


    target = 'duration'
    y_train = df_train[target].values
    y_val = df_val[target].values
    
    return X_train, X_val, y_train, y_val, dv






####################################
###  Modelling
####################################


def train_model_search(train, valid, y_val):

    def objective(params):
        with mlflow.start_run():
            mlflow.set_tag("model", "xgboost")
            mlflow.log_params(params)
            booster = xgb.train(
                params=params,
                dtrain=train,
                num_boost_round=1000,
                evals=[(valid, 'validation')],
                early_stopping_rounds=50
            )
            y_pred = booster.predict(valid)
            rmse = mean_squared_error(y_val, y_pred, squared=False)
            mlflow.log_metric("rmse", rmse)

        return {'loss': rmse, 'status': STATUS_OK}


    search_space = {
        'max_depth': scope.int(hp.choice('max_depth', np.arange(4, 100, 1))),
        'learning_rate': hp.loguniform('learning_rate', -3, 0),
        'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
        'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
        'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
        'objective': 'reg:squarederror',
        'seed': 42
    }

    best_result = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=1,
        trials=Trials()
    )
    
    print(best_result)

    
    return best_result

def train_best_model(train, valid, y_val, best_params, dv):
    with mlflow.start_run():
        
        best_params['seed'] = 42
        best_params['objective'] = 'reg:squarederror'


    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

In [18]:
train_path='data/green_tripdata_2021-01.parquet'
val_path='data/green_tripdata_2021-02.parquet'
X_train, X_val, y_train, y_val, dv = add_features(train_path, val_path)
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)
best_result = train_model_search(train, valid, y_val)

Training data: 73908
 Testing data: 61921
[0]	validation-rmse:18.69466                         
[1]	validation-rmse:16.58735                         
[2]	validation-rmse:14.81410                         
[3]	validation-rmse:13.33081                         
[4]	validation-rmse:12.09376                         
[5]	validation-rmse:11.07314                         
[6]	validation-rmse:10.23238                         
[7]	validation-rmse:9.54337                          
[8]	validation-rmse:8.98477                          
[9]	validation-rmse:8.53190                          
[10]	validation-rmse:8.16535                         
[11]	validation-rmse:7.86988                         
[12]	validation-rmse:7.63005                         
[13]	validation-rmse:7.44155                         
[14]	validation-rmse:7.28760                         
[15]	validation-rmse:7.16311                         
[16]	validation-rmse:7.05955                         
[17]	validation-rmse:6.97857            

In [19]:
best_result

{'learning_rate': 0.1408929636551727,
 'max_depth': 20,
 'min_child_weight': 0.40108842755325586,
 'reg_alpha': 0.04871389707067656,
 'reg_lambda': 0.08296953286308431}

In [20]:
train_best_model(train, valid, y_val, best_result, dv)

[0]	validation-rmse:18.69496
[1]	validation-rmse:16.58732
[2]	validation-rmse:14.81429
[3]	validation-rmse:13.33020
[4]	validation-rmse:12.09496
[5]	validation-rmse:11.07487
[6]	validation-rmse:10.23348
[7]	validation-rmse:9.54713
[8]	validation-rmse:8.98652
[9]	validation-rmse:8.53461
[10]	validation-rmse:8.16857
[11]	validation-rmse:7.87452
[12]	validation-rmse:7.63605
[13]	validation-rmse:7.44514
[14]	validation-rmse:7.29254
[15]	validation-rmse:7.16616
[16]	validation-rmse:7.06755
[17]	validation-rmse:6.98544
[18]	validation-rmse:6.92037
[19]	validation-rmse:6.86576
[20]	validation-rmse:6.82124
[21]	validation-rmse:6.78567
[22]	validation-rmse:6.75374
[23]	validation-rmse:6.72873
[24]	validation-rmse:6.70715
[25]	validation-rmse:6.68966
[26]	validation-rmse:6.67519
[27]	validation-rmse:6.66253
[28]	validation-rmse:6.65091
[29]	validation-rmse:6.64156
[30]	validation-rmse:6.63370
[31]	validation-rmse:6.62658
[32]	validation-rmse:6.62068
[33]	validation-rmse:6.61534
[34]	validation-r